In [1]:
from pyspark import SparkContext
from pyspark.sql.types import *
sc=spark.sparkContext

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1596372211575_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
lines = sc.textFile("s3://*/kddcup_data.txt")# use your S3 bucket
rdd = lines.map(lambda line: line.split(","))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
rdd.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['0', 'tcp', 'http', 'SF', '215', '45076', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0', '0', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', 'normal.']]

In [4]:
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import IndexToString

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
vectorized_rdd=rdd.map(lambda x: [x[-1], Vectors.dense(x[4:41])]).toDF(['label','features'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
vectorized_rdd.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(label='normal.', features=DenseVector([215.0, 45076.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))]

In [7]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(vectorized_rdd)
featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(vectorized_rdd)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
(trainingData, testData) = vectorized_rdd.randomSplit([0.8, 0.2])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=500)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",labels=labelIndexer.labels)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
model = pipeline.fit(trainingData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
predictions = model.transform(testData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
predictions.select("predictedLabel", "label", "features").show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|       normal.|back.|[41772.0,2920.0,0...|
|       normal.|back.|[50248.0,8314.0,0...|
|       normal.|back.|[53168.0,7300.0,0...|
|       normal.|back.|[54020.0,7300.0,0...|
|       normal.|back.|[54540.0,2920.0,0...|
|       normal.|back.|[54540.0,7300.0,0...|
|       normal.|back.|[54540.0,8314.0,0...|
|       normal.|back.|[54540.0,8314.0,0...|
|       normal.|back.|[54540.0,8314.0,0...|
|       normal.|back.|[54540.0,8314.0,0...|
|       normal.|back.|[54540.0,8314.0,0...|
|         back.|back.|[54540.0,8314.0,0...|
|         back.|back.|[54540.0,8314.0,0...|
|         back.|back.|[54540.0,8314.0,0...|
|         back.|back.|[54540.0,8314.0,0...|
|         back.|back.|[54540.0,8314.0,0...|
|         back.|back.|[54540.0,8314.0,0...|
|         back.|back.|[54540.0,8314.0,0...|
|         back.|back.|[54540.0,8314.0,0...|
|         back.|back.|[54540.0,8

In [15]:
predictions.select("predictedLabel", "label", "features").head(20)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(predictedLabel='normal.', label='back.', features=DenseVector([41772.0, 2920.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 6.0, 0.0, 0.0, 0.17, 0.17, 1.0, 0.0, 0.0, 255.0, 255.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05])), Row(predictedLabel='normal.', label='back.', features=DenseVector([50248.0, 8314.0, 0.0, 0.0, 0.0, 2.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 11.0, 0.0, 0.0, 0.1, 0.18, 1.0, 0.0, 0.18, 119.0, 119.0, 1.0, 0.0, 0.01, 0.0, 0.0, 0.0, 0.05, 0.05])), Row(predictedLabel='normal.', label='back.', features=DenseVector([53168.0, 7300.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.2, 0.2, 1.0, 0.0, 0.0, 255.0, 255.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06, 0.06])), Row(predictedLabel='normal.', label='back.', features=DenseVector([54020.0, 7300.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 7.0, 0.0, 0.0, 0.14